# **Task 1**

In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from numpy.linalg import inv
import time
import sys
import zlib

In [2]:
def dct_coeff():
    T = np.zeros([8, 8])
    for i in range(8):
        for j in range(8):
            if i == 0:
                T[i, j] = 1/np.sqrt(8)
            elif i > 0:
                T[i, j] = np.sqrt(2/8)*np.cos((2*j+1)*i*np.pi/16)
    return T


In [3]:
def quantization_level(n):
    Q50 = np.zeros([8, 8])

    Q50 = np.array([[16, 11, 10, 16, 24, 40, 52, 61],
                    [12, 12, 14, 19, 26, 58, 60, 55],
                    [14, 13, 16, 24, 40, 57, 69, 56],
                    [14, 17, 22, 29, 51, 87, 80, 62],
                    [18, 22, 37, 56, 68, 109, 103, 77],
                    [24, 35, 55, 64, 81, 104, 113, 92],
                    [49, 64, 78, 87, 103, 121, 120, 101],
                    [72, 92, 95, 98, 112, 100, 103, 99]])

    Q = np.zeros([8, 8])
    for i in range(8):
        for j in range(8):
            if n > 50:
                Q[i, j] = min(np.round((100-n)/50*Q50[i, j]), 255)
            else:
                Q[i, j] = min(np.round(50/n * Q50[i, j]), 255)
    return Q

In [4]:
def quantiz_div(a, b):
    tmp = np.zeros(a.shape)
    for i in range(8):
        for j in range(8):
            tmp[i, j] = np.round(a[i, j]/b[i, j])
    return tmp

In [5]:
def quantiz(D, Q):
    tmp = np.zeros(D.shape)
    mask = np.zeros([8, 8])
    for i in range(D.shape[0]//8):
        for j in range(D.shape[1]//8):
            mask = quantiz_div(D[8*i:8*i+8, 8*j:8*j+8], Q)
            tmp[8*i:8*i+8, 8*j:8*j+8] = mask
    return (tmp)

In [6]:
def decompress_mul(a, b):
    tmp = np.zeros(a.shape)
    for i in range(8):
        for j in range(8):
            tmp[i, j] = a[i, j]*b[i, j]
    return tmp

In [7]:
def decompress(C, Q, T, T_prime):
    R = np.zeros(C.shape)
    mask = np.zeros([8, 8])
    for i in range(C.shape[0]//8):
        for j in range(C.shape[1]//8):
            mask = decompress_mul(C[8*i:8*i+8, 8*j:8*j+8], Q)
            R[8*i:8*i+8, 8*j:8*j+8] = mask

    N = np.zeros(C.shape)

    for i in range(R.shape[0]//8):
        for j in range(R.shape[1]//8):
            mask = T_prime @ R[8*i:8*i+8, 8*j:8*j+8] @ T
            N[8*i:8*i+8, 8*j:8*j+8] = np.round(mask) + 128*np.ones([8, 8])

    return N

In [8]:
def Compress_img(file, level):

    I = cv2.imread(file)
    I = cv2.cvtColor(I,cv2.COLOR_BGR2YCrCb)
    B, G, R = cv2.split(I)

    H = I.shape[0]
    W = I.shape[1]

    print("Image size: ", I.shape)

    B = B - 128*np.ones([H, W])
    G = G - 128*np.ones([H, W])
    R = R - 128*np.ones([H, W])

    T = dct_coeff()
    T_prime = inv(T)
    Q = quantization_level(level)

    D_R = dct(R, T, T_prime)
    D_G = dct(G, T, T_prime)
    D_B = dct(B, T, T_prime)

    tmp = cv2.merge((D_B, D_G, D_R))

    cv2.imwrite('DCT.jpg', tmp)

    C_R = quantiz(D_R, Q)
    C_R[C_R == 0] = 0
    C_G = quantiz(D_G, Q)
    C_G[C_G == 0] = 0
    C_B = quantiz(D_B, Q)
    C_B[C_B == 0] = 0

    tmp = cv2.merge((C_B, C_G, C_R))
    C_B_str=zlib.compress(C_B.astype(np.int8).tobytes())
    C_G_str=zlib.compress(C_G.astype(np.int8).tobytes())
    C_R_str=zlib.compress(C_R.astype(np.int8).tobytes())
    return C_B_str, C_G_str, C_R_str, Q, T, T_prime,I.shape


In [9]:
def Decompress_img(C_B, C_G, C_R, Q, T, T_prime,shape):
    C_B=np.frombuffer(zlib.decompress(C_B),dtype=np.int8).astype(float).reshape(shape[0],shape[1])
    C_G=np.frombuffer(zlib.decompress(C_G),dtype=np.int8).astype(float).reshape(shape[0],shape[1])
    C_R=np.frombuffer(zlib.decompress(C_R),dtype=np.int8).astype(float).reshape(shape[0],shape[1])
    N_R = decompress(C_R, Q, T, T_prime)
    N_G = decompress(C_G, Q, T, T_prime)
    N_B = decompress(C_B, Q, T, T_prime)

    N_I = cv2.merge((N_B, N_G, N_R))
    N_I = cv2.cvtColor(cv2.convertScaleAbs(N_I),cv2.COLOR_YCR_CB2BGR)
    cv2.imwrite('Decompressed_task2.jpg', N_I)

In [10]:

def dct(M,T,T_prime):
    dct_res = np.zeros(M.shape)
    mask = np.zeros([8,8])
    for i in range(M.shape[0]//8):
        for j in range(M.shape[1]//8):
            mask = M[8*i:8*i+8,8*j:8*j+8]
            dct_res[8*i:8*i+8,8*j:8*j+8] = T @ mask @ T_prime

    return (dct_res)

In [11]:

file = "images/urban0.png"
level = 50

I = cv2.imread(file)

# change image to ycrcb
I = cv2.cvtColor(I, cv2.COLOR_BGR2YCrCb)

B, G, R = cv2.split(I)

H = I.shape[0]
W = I.shape[1]
shape = I.shape

print("Image size: ", I.shape)

B = B - 128*np.ones([H, W])
G = G - 128*np.ones([H, W])
R = R - 128*np.ones([H, W])

T = dct_coeff()
T_prime = inv(T)
Q = quantization_level(level)

D_R = dct(R, T, T_prime)
D_G = dct(G, T, T_prime)
D_B = dct(B, T, T_prime)

tmp = cv2.merge((D_B, D_G, D_R))

C_R = quantiz(D_R, Q)
C_R[C_R == 0] = 0
C_G = quantiz(D_G, Q)
C_G[C_G == 0] = 0
C_B = quantiz(D_B, Q)
C_B[C_B == 0] = 0

tmp = cv2.merge((C_B, C_G, C_R))

C_B_str = zlib.compress(C_B.astype(np.int8).tobytes())
C_G_str = zlib.compress(C_G.astype(np.int8).tobytes())
C_R_str = zlib.compress(C_R.astype(np.int8).tobytes())


# compression_percentage = 100*(sys.getsizeof(cv2.imread(file)) - (sys.getsizeof(C_B) + sys.getsizeof(C_G) + sys.getsizeof(C_R) + sys.getsizeof(level) + sys.getsizeof(shape))) / sys.getsizeof(cv2.imread(file))
# print(f"Original image size: {sys.getsizeof(cv2.imread(file))} bytes")
# print(f"Compressed image size: {sys.getsizeof(C_B) + sys.getsizeof(C_G) + sys.getsizeof(C_R) + sys.getsizeof(level) + sys.getsizeof(shape)} ({compression_percentage:0.2f}% smaller)")


C_B = np.frombuffer(zlib.decompress(C_B_str), dtype=np.int8).astype(float).reshape(I.shape[0], I.shape[1])
C_G = np.frombuffer(zlib.decompress(C_G_str), dtype=np.int8).astype(float).reshape(I.shape[0], I.shape[1])
C_R = np.frombuffer(zlib.decompress(C_R_str), dtype=np.int8).astype(float).reshape(I.shape[0], I.shape[1])

T = dct_coeff()
T_prime = inv(T)
Q = quantization_level(level)
N_R = decompress(C_R, Q, T, T_prime)
N_G = decompress(C_G, Q, T, T_prime)
N_B = decompress(C_B, Q, T, T_prime)

N_I = cv2.merge((N_B, N_G, N_R))

# change image to rgb
N_I = cv2.cvtColor(cv2.convertScaleAbs(N_I), cv2.COLOR_YCrCb2BGR)
cv2.imwrite('Decompressed_Task1.jpg', N_I)


Image size:  (256, 256, 3)


True

In [18]:
file = 'images/urban0.png'
level = 15
print("Filename: ", file)
print("Level of compression: ", level)

print("Compressing....")
start = time.time()
C_B, C_G, C_R, Q, T, T_prime,shape = Compress_img(file, level)
time_comp = time.time()
print("Compression Time: ", np.round(time_comp - start, 1), " sec")

print("Decompressing...")
Decompress_img(C_B, C_G, C_R, Q, T, T_prime,shape)
time_decomp = time.time()

print("Decompression Time: ", np.round(time_decomp - time_comp, 1), " sec")

end = time.time()
print("Total: ", np.round(end - start, 1), " sec")

Filename:  images/urban0.png
Level of compression:  15
Compressing....
Image size:  (256, 256, 3)
Compression Time:  0.6  sec
Decompressing...
Decompression Time:  0.1  sec
Total:  0.7  sec


In [20]:
import os

file_size = os.path.getsize('images/urban0.png')
image_size = os.path.getsize('Decompressed_task2.jpg')

print("File size:", file_size, "bytes")
print("Image size:", image_size, "bytes")
print("Compressed %: ", np.round(100*(file_size-image_size)/file_size, 2))

File size: 50082 bytes
Image size: 16732 bytes
Compressed %:  66.59
